# FFT를 이용한 분할정복으로 다항식의 곱을 $\nlog n$으로 풀기

## 1. 다항식의 표현

다항식을 표현하는 방법은 두 가지가 있다.

### (1) 계수 표현 (Coefficient Representation)

다항식을 계수의 나열로 표현하는 방식이다.  
차수가 낮은 항부터 오름차순으로 정리하면 다음과 같이 나타낼 수 있다:

$$(a_0, a_1, a_2, \ldots, a_d)$$

이때, 이 표현은 다음과 같은 다항식을 의미한다:

$$
f(x) = a_0 + a_1x + a_2x^2 + \cdots + a_dx^d
$$

### (2) 점값 표현 (Value Representation)

다항식 위의 $d+1$개의 점 $(x_i, f(x_i))$를 저장하는 방식이다.  
이 점값 표현의 가장 큰 특징은 다음과 같다:

> **$d+1$개의 서로 다른 점을 모두 지나는 $d$차 다항식은 오직 하나만 존재한다.**

이 성질은 FFT를 기반으로 한 다항식 곱셈에서 핵심이 된다.

## 2. 다항식의 유일성 증명

$d=2$일 때를 예시로 들어 보자. 즉, 2차 다항식이 세 점을 지난다고 가정한다.  
이때 서로 다른 두 개의 2차 다항식 $P(x)$, $Q(x)$가 동일한 세 점 $(x_1, y_1), (x_2, y_2), (x_3, y_3)$을 지난다고 하자.

두 다항식의 차를 정의하면:

$$
R(x) = P(x) - Q(x)
$$

$R(x)$는 2차 이하의 다항식이다. 그런데:

$$
R(x_1) = R(x_2) = R(x_3) = 0
$$

즉, $R(x)$는 서로 다른 세 점에서 0이 된다.  
하지만 2차 이하의 다항식은 최대 두 개의 서로 다른 실근만 가질 수 있다.  
세 점에서 모두 0이 되는 것은 불가능하므로 모순이다.
따라서 서로 다른 $d+1$개의 점을 모두 지나는 $d$차 다항식은 유일하다.


In [ ]:
#fft 구현

import cmath 

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    result = []
    array2, array3 = [], []

    for i in range(n // 2): 
        array2.append(array[2 * i])
    for j in range(n // 2):
        array3.append(array[2 * j + 1]) 

    squared_half_solution = []
    for i in range(n // 2):
        squared_half_solution.append(input_solution[i] ** 2) 

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    for i in range(n // 2):
        result.append(P[i] + input_solution[i] * Q[i])
    for i in range(n // 2):
        result.append(P[i] - input_solution[i] * Q[i])
    
    return result

array = [1, 2, 3, 4, 5, 6, 7, 8]

n = len(array)
w_list = [cmath.exp(2j * cmath.pi * i / n) for i in range(n)]

result = fft(array, w_list[:n//2])
print(result)

print(w_list[0])

In [ ]:
#정확한 수식 구현

from sympy import symbols, exp, I, pi, Rational, simplify

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    result = []
    array2 = [array[2*i] for i in range(n//2)]
    array3 = [array[2*i+1] for i in range(n//2)]

    squared_half_solution = [simplify(w**2) for w in input_solution[:n//2]]

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    for i in range(n//2):
        result.append(simplify(P[i] + input_solution[i] * Q[i]))
    for i in range(n//2):
        result.append(simplify(P[i] - input_solution[i] * Q[i]))

    return result

# 입력
n = 8
w_list = [exp(2 * pi * I * i / n) for i in range(n)]
array = [Rational(x) for x in [1,2,3,4,5,6,7,8]]

# 실행
result = fft(array, w_list[:n//2])
for val in result:
    print(val)